In [64]:
from datetime import timedelta

from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
from snowflake.core import Root, CreateMode
from snowflake.core.database import Database
from snowflake.core.schema import Schema
from snowflake.core.stage import Stage
from snowflake.core.table import Table, TableColumn, PrimaryKey
from snowflake.core.task import StoredProcedureCall, Task
from snowflake.core.task.dagv1 import DAGOperation, DAG, DAGTask
from snowflake.core.warehouse import Warehouse

session = Session.builder.configs({"connection_name": "default"}).create()

root = Root(session)


In [46]:
from snowflake.core.database import Database
from snowflake.core.schema import Schema

database = root.databases.create(Database(name="spcs_python_api_db"), mode="orreplace")
schema = database.schemas.create(Schema(name="public"), mode="orreplace")

In [66]:
from snowflake.core.image_repository import ImageRepository

my_repo = ImageRepository("MyImageRepository")
schema.image_repositories.create(my_repo, mode=CreateMode.if_not_exists)

<ImageRepositoryResource: 'spcs_python_api_db.public.MyImageRepository'>

In [67]:
my_repo_res = schema.image_repositories["MyImageRepository"]
my_repo = my_repo_res.fetch()
print(my_repo.name)

MYIMAGEREPOSITORY


In [73]:
repositories = schema.image_repositories
for repo_obj in repositories.iter():
    print(repo_obj.repository_url)

wmrfcux-mi66915.registry.snowflakecomputing.com/spcs_python_api_db/public/myimagerepository


In [69]:
from snowflake.core.compute_pool import ComputePool

new_compute_pool_def = ComputePool(
    name="MyComputePool",
    instance_family="CPU_X64_XS",
    min_nodes=1,
    max_nodes=2,
)

new_compute_pool = root.compute_pools.create(new_compute_pool_def, mode=CreateMode.if_not_exists)

In [70]:
image_repository = schema.image_repositories["MyImageRepository"]

In [75]:
from textwrap import dedent
from io import BytesIO
from snowflake.core.service import Service, ServiceSpecInlineText

specification = dedent(f"""\
    spec:
      containers:
      - name: web-server
        image: wmrfcux-mi66915.registry.snowflakecomputing.com/spcs_python_api_db/public/myimagerepository:latest
      endpoints:
      - name: public-access
        port: 80
        protocol: HTTP
        public: true
    """)

service_def = Service(
    name="MyService",
    compute_pool="MyComputePool",
    spec=ServiceSpecInlineText(spec_text=specification),
    min_instances=1,
    max_instances=1,
)

nginx_service = schema.services.create(service_def, mode=CreateMode.if_not_exists)

In [76]:
from pprint import pprint

print(nginx_service.get_service_status(timeout=5))

pprint(nginx_service.get_service_status(timeout=5))

[{'status': 'PENDING', 'message': 'Unschedulable due to insufficient  resources.', 'containerName': 'web-server', 'instanceId': '0', 'serviceName': 'MYSERVICE', 'image': 'wmrfcux-mi66915.registry.snowflakecomputing.com/spcs_python_api_db/public/myimagerepository:latest', 'restartCount': 0, 'startTime': ''}]
[{'containerName': 'web-server',
  'image': 'wmrfcux-mi66915.registry.snowflakecomputing.com/spcs_python_api_db/public/myimagerepository:latest',
  'instanceId': '0',
  'message': 'Unschedulable due to insufficient  resources.',
  'restartCount': 0,
  'serviceName': 'MYSERVICE',
  'startTime': '',
  'status': 'PENDING'}]


In [30]:
# import json, time

# while True:
#     public_endpoints = (await nginx_service.fetch()).public_endpoints # this doesn't work
#     try:
#         endpoints = json.loads(public_endpoints)
#     except json.JSONDecodeError:
#         print(public_endpoints)
#         time.sleep(15)
#     else:
#         break

TypeError: object Service can't be used in 'await' expression

In [94]:
import time

svc_fqn = f"{database.name}.{schema.name}.MYSERVICE"  # match your actual service name

while True:
    rows = session.sql(f"SHOW ENDPOINTS IN SERVICE {svc_fqn}").collect()
    # rows are Row objects, inspect available fields if needed:
    # print(rows[0].as_dict())
    public = [r.as_dict() for r in rows if r.as_dict().get("is_public")]
    if public and public[0].get("ingress_url"):
        print("Public URL:", public[0]["ingress_url"])
        break
    print("Endpoints not ready yet. Current:", [r.as_dict() for r in rows])
    time.sleep(15)
endpoints = public[0]["ingress_url"]


Public URL: mbb4el-wmrfcux-mi66915.snowflakecomputing.app


In [92]:
session.sql("SHOW COMPUTE POOLS LIKE 'MYCOMPUTEPOOL'").show()


-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"name"         |"state"  |"min_nodes"  |"max_nodes"  |"instance_family"  |"num_services"  |"num_jobs"  |"auto_suspend_secs"  |"auto_resume"  |"active_nodes"  |"idle_nodes"  |"target_nodes"  |"created_on"                      |"resumed_on"                      |"updated_on"                      |"owner"       |"comment"  |"is_exclusive"  |"application"  |"placement_group"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [95]:
import webbrowser

# print(f"Visiting {endpoints['public-access']} in your browser. You might need to log in there.")
webbrowser.open(f"https://{endpoints}")

True

In [96]:
from time import sleep

nginx_service.suspend()
sleep(3)
pprint(nginx_service.get_service_status(timeout=5))

[{'containerName': 'web-server',
  'image': 'wmrfcux-mi66915.registry.snowflakecomputing.com/spcs_python_api_db/public/myimagerepository:latest',
  'instanceId': '0',
  'message': 'Terminating.',
  'restartCount': 0,
  'serviceName': 'MYSERVICE',
  'startTime': '2026-01-27T04:51:43Z',
  'status': 'TERMINATING'}]


In [97]:
new_compute_pool.suspend()
# nginx_service.suspend()

# new_compute_pool_def
nginx_service.drop()

In [57]:
session.sql("SHOW COMPUTE POOLS").show()
session.sql(
    f"SHOW SERVICES IN SCHEMA {database.name}.{schema.name}"
).show()


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"name"                   |"state"    |"min_nodes"  |"max_nodes"  |"instance_family"  |"num_services"  |"num_jobs"  |"auto_suspend_secs"  |"auto_resume"  |"active_nodes"  |"idle_nodes"  |"target_nodes"  |"created_on"                      |"resumed_on"                      |"updated_on"                      |"owner"       |"comment"                       |"is_exclusive"  |"application"  |"placement_group"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [58]:
nginx_service.resume()
sleep(3)
print(nginx_service.get_service_status(timeout=5))

NotFoundError: (404)
Reason: Not Found
Error Message: service 'spcs_python_api_db.public.myservice' does not exist or not authorized.
HTTP response code: 404
Request ID: d880aede-58ba-4faa-b9a9-3e8f1ae2d0bd
Error Code: 002003